## Have Customers Narrow Their Travel Searches Based on Temperature and Precipitation

In [1]:
import pandas as pd
import os
import re
from config import g_key
import requests
import gmaps

Import the WeatherPy_vacation.csv file from Part 1 as a new DataFrame

In [2]:
# Output file
infolder = "weather_data"
infile = "WeatherPy_challenge.csv"

vacation_df = pd.read_csv(os.path.join(infolder,infile))
vacation_df

,Unnamed: 0,Name,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches)
0,0,Evensk,RU,61.95,159.23,36.68,80,75,3.98,broken clouds,0.0,0.0
1,1,Barabinsk,RU,55.36,78.36,64.36,60,82,3.06,broken clouds,0.0,0.0
2,2,Lárdos,GR,36.09,28.02,73.40,49,0,16.11,clear sky,0.0,0.0
3,3,Kawalu,ID,-7.38,108.21,74.48,91,29,0.72,scattered clouds,0.0,0.0
4,4,Porto Velho,BR,-8.76,-63.90,68.00,68,75,5.82,broken clouds,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
584,584,Kaduqli,SD,11.02,29.72,97.83,26,66,2.89,broken clouds,0.0,0.0
585,585,Bilma,NE,18.69,12.92,108.01,8,99,12.46,overcast clouds,0.0,0.0
586,586,Grand-Lahou,CI,5.14,-5.02,83.43,76,19,12.03,few clouds,0.0,0.0
587,587,Bukachacha,RU,52.98,116.92,47.25,54,70,4.03,broken clouds,0.0,0.0


Filter the DataFrame for minimum and maximum temperature preferences, and if the rain or snow accumulation is 0 inches or not using conditional statements.

In [3]:
# Constants
yesno_regex = "^(yes|no)$"

In [4]:
#Prompt the customer for the minimum temperature preference.

while True:
    try:
        min_str = input("\nWhat is the minimum temperature you would like for your trip? ")
        pref_temp_min = float(min_str)
        break
    except:
        print(f"\n<{min_str}> is an invalid temperature value. Please try again.")
        pass
    
#Prompt the customer for the maximum temperature preference.
while True:
    try:
        max_str = input("\nWhat is the maximum temperature you would like for your trip? ")
        pref_temp_max = float(max_str)
        break
    except:
        print(f"\n<{max_str}> is an invalid temperature value. Please try again.")
        pass
   


What is the minimum temperature you would like for your trip? 75

What is the maximum temperature you would like for your trip? 90


In [5]:
# Prompt the customer to answer if he or she would like it to be raining or not

while True: 
    raining = input("Do you want it to be raining? (yes/no) ")
    raining = raining.lower()
    if re.search(yesno_regex, raining):
        raining = raining == "yes"
        break
    else:
        print("Please answer yes or no.")
    

Do you want it to be raining? (yes/no) no


In [6]:
# Prompt the customer to answer if he or she would like it to be snowing or not

while True: 
    snowing = input("Do you want it to be snowing? (yes/no) ")
    snowing = snowing.lower()
    if re.search(yesno_regex, snowing):
        snowing = snowing == "yes"
        break
    else:
        print("Please answer yes or no.")

Do you want it to be snowing? (yes/no) no


In [50]:
# Filter dataframe 
filtered_df = vacation_df.loc[(vacation_df["Max Temp"] <= pref_temp_max) &
                              (vacation_df["Max Temp"] >= pref_temp_min) ].copy().reset_index()
if raining:
    filtered_df = filtered_df[filtered_df["Rain (inches)"] > 0]
else:
    filtered_df = filtered_df[filtered_df["Rain (inches)"] == 0]
    
if snowing:
    filtered_df = filtered_df[filtered_df["Snow (inches)"] > 0]
else:
    filtered_df = filtered_df[filtered_df["Snow (inches)"] == 0]

In [51]:
filtered_df

,index,Unnamed: 0,Name,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches)
0,5,5,Vanimo,PG,-2.67,141.30,80.83,80,99,2.82,light rain,0.0,0.0
1,7,7,Salalah,OM,17.02,54.09,87.80,79,75,9.17,broken clouds,0.0,0.0
2,11,11,Sur,OM,22.57,59.53,87.28,65,51,16.98,broken clouds,0.0,0.0
3,12,12,Sabang,ID,5.89,95.32,80.19,82,100,11.43,light rain,0.0,0.0
4,15,15,Bambanglipuro,ID,-7.95,110.28,79.23,82,1,2.13,clear sky,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,569,569,Ko Samui,TH,9.54,99.94,86.00,79,20,2.24,few clouds,0.0,0.0
200,570,570,Praia da Vitória,PT,38.73,-27.07,75.20,64,40,12.95,scattered clouds,0.0,0.0
201,572,572,Kapit,MY,2.02,112.93,76.42,94,100,1.68,moderate rain,0.0,0.0
202,573,573,Sarankhola,BD,22.31,89.79,83.05,79,55,11.92,broken clouds,0.0,0.0


In [63]:
def get_hotel(lat,lng):
    
    # Use base URL to search for hotels
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # params ... missing location
    params_temp = {"radius": 5000,
    "types": "lodging",
    "key": g_key}
    
    hotels = []
    
    for index in range(len(lat)):
        params = params_temp
        params["location"] = f"{lat[index]},{lng[index]}"
    
        # Make request and get the JSON data from the search.
        hotels.append(requests.get(base_url, params=params).json())
        
    return hotels

In [91]:
hotels1 = get_hotel([-2.67, 17.02], [141.30,54.09])
hotels2 = pd.Series(hotels1)
hotel_names = []
for hotel in hotels2:
    #print(hotel["results"][0]["name"])
    hotel_name = hotel["results"][0]["name"]
    hotel_names.append(hotel_name)


In [106]:
loc = filtered_df[["Lat","Lng"]]
#filtered_df["Hotel Name"] = get_hotel(filtered_df["Lat"],filtered_df["Lng"])
hotels1 = get_hotel(filtered_df["Lat"],filtered_df["Lng"])

hotels2 = pd.Series(hotels1)
hotel_names = []
len(hotels2)
for hotel in hotels2:
    try:
        hotel_name = hotel["results"][0]["name"]
        hotel_names.append(hotel_name)
    except:
        hotel_names.append("NA")
        
filtered_df["Hotel_Name"] = hotel_names
filtered_df

,index,Unnamed: 0,Name,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches),Hotel_Name
0,5,5,Vanimo,PG,-2.67,141.30,80.83,80,99,2.82,light rain,0.0,0.0,Vahmoneh Lodge Vanimo
1,7,7,Salalah,OM,17.02,54.09,87.80,79,75,9.17,broken clouds,0.0,0.0,HAMDAN PLAZA HOTEL
2,11,11,Sur,OM,22.57,59.53,87.28,65,51,16.98,broken clouds,0.0,0.0,Sur Plaza Hotel
3,12,12,Sabang,ID,5.89,95.32,80.19,82,100,11.43,light rain,0.0,0.0,Hotel Citra
4,15,15,Bambanglipuro,ID,-7.95,110.28,79.23,82,1,2.13,clear sky,0.0,0.0,Puri Brata
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,569,569,Ko Samui,TH,9.54,99.94,86.00,79,20,2.24,few clouds,0.0,0.0,"The Passage Samui Villas & Resort, Thailand"
200,570,570,Praia da Vitória,PT,38.73,-27.07,75.20,64,40,12.95,scattered clouds,0.0,0.0,Hotel Branco I
201,572,572,Kapit,MY,2.02,112.93,76.42,94,100,1.68,moderate rain,0.0,0.0,Hotel Meligai Kapit
202,573,573,Sarankhola,BD,22.31,89.79,83.05,79,55,11.92,broken clouds,0.0,0.0,মুক্তিযোদ্ধা শ্রী শ্রী মৃনাল হালদার


In [101]:
hotels2[0]["results"][0]["name"]

'Vahmoneh Lodge Vanimo'

In [105]:
for hotel in hotels2:
    print(hotel)
    try:
        hotel_name = hotel["results"][0]["name"]
        hotel_names.append(hotel_name)
    except:
        hotel_names.append("NA")
        
hotel_names

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



['Vahmoneh Lodge Vanimo',
 'HAMDAN PLAZA HOTEL',
 'Sur Plaza Hotel',
 'Hotel Citra',
 'Puri Brata',
 'Amaloh Boutique Resort',
 'Lago',
 'Pension Oaoa',
 'Hotel Adriatico',
 'Casa Hotel Victoria',
 'Guest Hotel',
 'Grand Port Royal Hotel Marina & Spa',
 'Swiss Belhotel Manokwari',
 'HS Hotsson Smart Acapulco',
 'Hôtel Abda',
 'Minshuku Agaihama',
 'Vahmoneh Lodge Vanimo',
 'HAMDAN PLAZA HOTEL',
 'Sur Plaza Hotel',
 'Hotel Citra',
 'Puri Brata',
 'Amaloh Boutique Resort',
 'Lago',
 'Pension Oaoa',
 'Hotel Adriatico',
 'Casa Hotel Victoria',
 'Guest Hotel',
 'Grand Port Royal Hotel Marina & Spa',
 'Swiss Belhotel Manokwari',
 'HS Hotsson Smart Acapulco',
 'Hôtel Abda',
 'Minshuku Agaihama',
 'Vahmoneh Lodge Vanimo',
 'HAMDAN PLAZA HOTEL',
 'Sur Plaza Hotel',
 'Hotel Citra',
 'Puri Brata',
 'Amaloh Boutique Resort',
 'Lago',
 'Pension Oaoa',
 'Hotel Adriatico',
 'Casa Hotel Victoria',
 'Guest Hotel',
 'Grand Port Royal Hotel Marina & Spa',
 'Swiss Belhotel Manokwari',
 'HS Hotsson Smart A